# Realizando ingeniería de datos para realizar transformación y generación de nuevas variables

## 1.- Importamos las librerías necesarias y preparamos las funciones oportunas.

In [1]:
import pandas as pd
import numpy as np
import os
import pickle
import re
import spacy
from nltk.stem.snowball import SnowballStemmer
import datetime

nlp = spacy.load('es_core_news_lg')

In [2]:
pd.set_option('display.max_colwidth', None)       # Visualizar todas las columnas
pd.set_option('display.max_rows', None)           # Visualizar todas las filas

### Obtenemos los datos guardados.

In [3]:
path=r'Data\productos_scrape.csv'

In [5]:
os.chdir(os.path.split(os.getcwd())[0])
folder=os.getcwd()
folder

'd:\\Data_science\\Javier\\Repositorios\\Proyecto_tienda_online_webscrapping'

In [6]:
new_path=folder+'\\'+path
new_path

'd:\\Data_science\\Javier\\Repositorios\\Proyecto_tienda_online_webscrapping\\Data\\productos_scrape.csv'

Vamos a eliminar duplicados del dataframe (esto es una operación a realizar en el anterior Notebook)

In [7]:
dataframe=pd.read_csv(new_path)
dataframe.head()

,ID,NAME,INFO,LISTA_URL,REGULAR_PRICE,DISCOUNT_PRICE
0,0,Desliz! Lubricante íntimo de agua 100ml,"Algunos lubricantes son un poco densos, otros muy líquidos, unos pringan, otros se absorben demasiado rápido, unos son muy caros, otros son de baja calidad... tras años trabajando con más de 15 marcas de lubricantes íntimos, en amantis y amantis.net nos hemos decidido a buscar la fórmula más equilibrada posible, creemos tenerla en nuestras manos...Desliz! de agua, es un lubricante íntimo concentrado y duradero. Por su densidad es ideal para el uso con juguetes eróticos de cualquier material, 100% compatible con preservativos, perfecto para el uso como lubricante íntimo vaginal. Complementa y facilita las relaciones sexuales.Sin olor ni sabor, no mancha, totalmente hipoalergénico y altamente hidratante dejará tu piel suave y deslizante incluso después de su total absorción. Ver características y medidasBote de 100ml de venta exclusiva en amantis.net y tiendas amantis.Ingredientes: Aquam glycerin, hydroxyethylcellulose, diazolidinyl urea, sodium benzoate, potassium sorbate, citric acid.",https://www.amantis.net/desliz-lubricante-intimo-agua-100ml/,9.99,7.99
1,1,"TOBOGANE HOT RABBIT, el superventas de amantis ¡mejorado!","Vuelve nuestro vibrador de doble estimulación más vendido de AMANTIS… ¡Ahora con rabbit!Si ya conoces a TOBOGANE, nuestro súperventas con doble motor, mango ergonómico gracias a su anilla y efecto calor, vas a enamorarte completamente de su versión más actualizada; le hemos añadido las orejitas a su brazo de estimulación externa y el resultado es un potente conejo que bordeará y hará trepidar a tu clítoris (o perineo) para una estimulación más completa, precisa y profunda.Esta nueva versión Rabbit, cuenta con 2 motores: uno en cada brazo de estimulación dual (interna y externa) con 10 patrones de vibración combinándolos de forma independiente y otro de efecto calor… que caldeará y templará a tu adorado juguete para tener una experiencia total HOT sin sobresaltos.Déjate caer por un tobogán de sensaciones y por las sinuosas curvas del nuevo TOBOGANE RABBIT… que está vez te hará también saltar. Ver características y medidasMedidas: 19cm (11cm insertables) y 3,3cm/ 2,2cm (ancho máximo/ mínimo)Material: Silicona médica + ABSVibración: 10 modos + 6 intensidadesEfecto calor independienteRecargable (cable USB incluido)Sumergible IPX7Función de bloqueo para viajesIncluye estuche rígido y bolsa de saténColor: púrpura o negroLimpieza y conservación: Recordamos que el uso de lubricante es aconsejable para cualquier actividad sexual (siempre con base de agua al utilizar juguetes de silicona). No olvides lavar tus juguetes antes y después de cada uso, con agua y jabón neutro o un limpiador específico de acción desinfectante y bactericida, como nuestro spray limpiador Limpín. Todos los juguetes deben estar separados entre sí, en un lugar fresco y seco; para ello puedes utilizar la bolsa de satén incluida o una de nuestras bolsas de terciopelo en diferentes tamaños.",https://www.amantis.net/tobogane-hot-rabbit-el-superventas-amantis-mejorado/,89.99,39.99
2,2,"BALLENATO, tu vibrador a distancia con aleta móvil y sumergible","De las profundidades más húmedas llega BALLENATO, el nuevo vibrador ergonómico de AMANTIS.Este grandioso cetáceo vibrante es poseedor de una colita estructurada que adquiere la forma que tú le des. De tal manera que podrías usarlo: como un potente dildo (dejando la aleta articulada en línea recta), como juguete de doble estimulación (vagina- clítoris / ano- perineo) o como vibrador tipo C (insertando sólo la parte de la aleta y permitiendo la penetración mientras lo llevas puesto)… entre muchas otras formas de estimulación externa.Además, BALLENATO es disfrutón y aparte de pasarlo muy bien en el hogar y recrearse en los juegos húmedos del baño, le encanta dar paseos al aire libre o incluso que lo saques a cenar porque gracias a su mando a distancia, podrás llevarlo puesto y activarlo allá donde tú quieras.Hazte 

In [7]:
len(dataframe)

551

## 2.- Descomponemos las variables *Name* y *Description*.

Vamos a crear 2 nuevas columnas a partir de *Name* y *Description*, donde dejaremos el nombre del producto y su slogan por un lado y por otro la descripción y las características por otro.

En principio son variables que no necesitaremos para estos estudios pero las guardaremos por si hay que hacer algún estudio posterior de las mismas (comparativas de productos similares de esta página web o de otras páginas web, por ejemplo).

#### Generación de slogan.

Primeramente vamos a realizar un visionado de los datos que tienen para discernir donde se puede realizar la separación.

Por un lado extraeremos el Name en una nueva columna y varias columnas para ir separando el resto. 

In [8]:
dataframe['NAME'] = dataframe['NAME'].str.replace(r'-(?=\w)', '_')
dataframe[['PRODUCT', 'SLOGAN']] = dataframe['NAME'].str.split('[,-.]', 1, expand=True)
dataframe['PRODUCT'] = dataframe['PRODUCT'].str.strip()
dataframe['SLOGAN'] = dataframe['SLOGAN'].str.strip()
# dataframe.head()

C:\Users\Javier\AppData\Local\Temp\ipykernel_13436\572804979.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  dataframe['NAME'] = dataframe['NAME'].str.replace(r'-(?=\w)', '_')
C:\Users\Javier\AppData\Local\Temp\ipykernel_13436\572804979.py:2: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  dataframe[['PRODUCT', 'SLOGAN']] = dataframe['NAME'].str.split('[,-.]', 1, expand=True)


#### Separación entre Description y Characteristics.

Primeramente vamos a realizar un visionado de los datos que tienen para discernir donde se puede realizar la separación.

Por un lado extraeremos las características en una nueva columna, dejando la Description en la misma. 

**Pendiente**: Hay que hacer una segunda separación de Description en el caso que solo aparezca *características*

In [9]:
dataframe['CHARACTERISTICS'] = dataframe['INFO'].str.split('Ver características y medidas|Características', 1).str[1]
dataframe['DESCRIPTION'] = dataframe['INFO'].str.split('Ver características y medidas|Características', 1).str[0].str.strip()
# dataframe.head()


C:\Users\Javier\AppData\Local\Temp\ipykernel_13436\1793188304.py:1: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  dataframe['CHARACTERISTICS'] = dataframe['INFO'].str.split('Ver características y medidas|Características', 1).str[1]
C:\Users\Javier\AppData\Local\Temp\ipykernel_13436\1793188304.py:2: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  dataframe['DESCRIPTION'] = dataframe['INFO'].str.split('Ver características y medidas|Características', 1).str[0].str.strip()


Se observan que hay *\r* dentro del texto generado en las nuevas columnas, por lo que vamos a proceder a sustituir esto por un espacio.

In [10]:
dataframe['CHARACTERISTICS'] = dataframe['CHARACTERISTICS'].str.replace('\r', ' ')
dataframe['DESCRIPTION'] = dataframe['DESCRIPTION'].str.replace('\r', ' ')
# dataframe.head()

In [11]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 550 entries, 0 to 549
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               550 non-null    int64  
 1   NAME             550 non-null    object 
 2   INFO             550 non-null    object 
 3   LISTA_URL        550 non-null    object 
 4   REGULAR_PRICE    550 non-null    float64
 5   DISCOUNT_PRICE   550 non-null    float64
 6   PRODUCT          550 non-null    object 
 7   SLOGAN           454 non-null    object 
 8   CHARACTERISTICS  489 non-null    object 
 9   DESCRIPTION      550 non-null    object 
dtypes: float64(2), int64(1), object(7)
memory usage: 43.1+ KB


Eliminamos *Nombre* y sustituimos por *Product*, *Slogan*, *Description* y *Characteristics*.

In [12]:
col_1 = dataframe.pop('PRODUCT')
col_2=dataframe.pop('SLOGAN')
col_3=dataframe.pop('DESCRIPTION')
col_4=dataframe.pop('CHARACTERISTICS')

dataframe.drop(columns=['NAME'],inplace=True)
dataframe.drop(columns=['INFO'],inplace=True)

dataframe.insert(loc= 1 , column= 'PRODUCT', value= col_1)
dataframe.insert(loc= 2 , column= 'SLOGAN', value= col_2)
dataframe.insert(loc= 3 , column= 'DESCRIPTION', value= col_3)
dataframe.insert(loc= 4 , column= 'CHARACTERISTICS', value= col_4)
dataframe.head()


# col_1 = dataframe.pop('Product')
# col_2=dataframe.pop('Slogan')
# col_3=dataframe.pop('Characteristics')
# dataframe.drop(columns=['Name'],inplace=True)

# dataframe.insert(loc= 0 , column= 'Product', value= col_1)
# dataframe.insert(loc= 1 , column= 'Slogan', value= col_2)
# dataframe.insert(loc= 3 , column= 'Characteristics', value= col_3)
# dataframe.head()

,ID,PRODUCT,SLOGAN,DESCRIPTION,CHARACTERISTICS,LISTA_URL,REGULAR_PRICE,DISCOUNT_PRICE
0,0,Desliz! Lubricante íntimo de agua 100ml,None,"Algunos lubricantes son un poco densos, otros muy líquidos, unos pringan, otros se absorben demasiado rápido, unos son muy caros, otros son de baja calidad... tras años trabajando con más de 15 marcas de lubricantes íntimos, en amantis y amantis.net nos hemos decidido a buscar la fórmula más equilibrada posible, creemos tenerla en nuestras manos...Desliz! de agua, es un lubricante íntimo concentrado y duradero. Por su densidad es ideal para el uso con juguetes eróticos de cualquier material, 100% compatible con preservativos, perfecto para el uso como lubricante íntimo vaginal. Complementa y facilita las relaciones sexuales.Sin olor ni sabor, no mancha, totalmente hipoalergénico y altamente hidratante dejará tu piel suave y deslizante incluso después de su total absorción.","Bote de 100ml de venta exclusiva en amantis.net y tiendas amantis.Ingredientes: Aquam glycerin, hydroxyethylcellulose, diazolidinyl urea, sodium benzoate, potassium sorbate, citric acid.",https://www.amantis.net/desliz-lubricante-intimo-agua-100ml/,9.99,7.99
1,1,TOBOGANE HOT RABBIT,el superventas de amantis ¡mejorado!,"Vuelve nuestro vibrador de doble estimulación más vendido de AMANTIS… ¡Ahora con rabbit!Si ya conoces a TOBOGANE, nuestro súperventas con doble motor, mango ergonómico gracias a su anilla y efecto calor, vas a enamorarte completamente de su versión más actualizada; le hemos añadido las orejitas a su brazo de estimulación externa y el resultado es un potente conejo que bordeará y hará trepidar a tu clítoris (o perineo) para una estimulación más completa, precisa y profunda.Esta nueva versión Rabbit, cuenta con 2 motores: uno en cada brazo de estimulación dual (interna y externa) con 10 patrones de vibración combinándolos de forma independiente y otro de efecto calor… que caldeará y templará a tu adorado juguete para tener una experiencia total HOT sin sobresaltos.Déjate caer por un tobogán de sensaciones y por las sinuosas curvas del nuevo TOBOGANE RABBIT… que está vez te hará también saltar.","Medidas: 19cm (11cm insertables) y 3,3cm/ 2,2cm (ancho máximo/ mínimo)Material: Silicona médica + ABSVibración: 10 modos + 6 intensidadesEfecto calor independienteRecargable (cable USB incluido)Sumergible IPX7Función de bloqueo para viajesIncluye estuche rígido y bolsa de saténColor: púrpura o negroLimpieza y conservación: Recordamos que el uso de lubricante es aconsejable para cualquier actividad sexual (siempre con base de agua al utilizar juguetes de silicona). No olvides lavar tus juguetes antes y después de cada uso, con agua y jabón neutro o un limpiador específico de acción desinfectante y bactericida, como nuestro spray limpiador Limpín. Todos los juguetes deben estar separados entre sí, en un lugar fresco y seco; para ello puedes utilizar la bolsa de satén incluida o una de nuestras bolsas de terciopelo en diferentes tamaños.",https://www.amantis.net/tobogane-hot-rabbit-el-superventas-amantis-mejorado/,89.99,39.99
2,2,BALLENATO,tu vibrador a distancia con aleta móvil y sumergible,"De las profundidades más húmedas llega BALLENATO, el nuevo vibrador ergonómico de AMANTIS.Este grandioso cetáceo vibrante es poseedor de una colita estructurada que adquiere la forma que tú le des. De tal manera que podrías usarlo: como un potente dildo (dejando la aleta articulada en línea recta), como juguete de doble estimulación (vagina- clítoris / ano- perineo) o como vibrador tipo C (insertando sólo la parte de la aleta y permitiendo la penetración mientras lo llevas puesto)… entre muchas otras formas de estimulación externa.Además, BALLENATO es disfrutón y aparte de pasarlo muy bien en el hogar y recrearse en los juegos húmedos del baño, le encanta dar paseos al aire libre o incluso que lo saques a cenar porque gracias a su mando a distancia, podrás llevarlo puesto y activarlo allá donde tú quieras.Hazte ya las veinte mil legu

Pasamos a un fichero .csv para guardarlo

In [13]:
df_engineer=dataframe.iloc[:,:4]
df_engineer.head()

,ID,PRODUCT,SLOGAN,DESCRIPTION
0,0,Desliz! Lubricante íntimo de agua 100ml,None,"Algunos lubricantes son un poco densos, otros muy líquidos, unos pringan, otros se absorben demasiado rápido, unos son muy caros, otros son de baja calidad... tras años trabajando con más de 15 marcas de lubricantes íntimos, en amantis y amantis.net nos hemos decidido a buscar la fórmula más equilibrada posible, creemos tenerla en nuestras manos...Desliz! de agua, es un lubricante íntimo concentrado y duradero. Por su densidad es ideal para el uso con juguetes eróticos de cualquier material, 100% compatible con preservativos, perfecto para el uso como lubricante íntimo vaginal. Complementa y facilita las relaciones sexuales.Sin olor ni sabor, no mancha, totalmente hipoalergénico y altamente hidratante dejará tu piel suave y deslizante incluso después de su total absorción."
1,1,TOBOGANE HOT RABBIT,el superventas de amantis ¡mejorado!,"Vuelve nuestro vibrador de doble estimulación más vendido de AMANTIS… ¡Ahora con rabbit!Si ya conoces a TOBOGANE, nuestro súperventas con doble motor, mango ergonómico gracias a su anilla y efecto calor, vas a enamorarte completamente de su versión más actualizada; le hemos añadido las orejitas a su brazo de estimulación externa y el resultado es un potente conejo que bordeará y hará trepidar a tu clítoris (o perineo) para una estimulación más completa, precisa y profunda.Esta nueva versión Rabbit, cuenta con 2 motores: uno en cada brazo de estimulación dual (interna y externa) con 10 patrones de vibración combinándolos de forma independiente y otro de efecto calor… que caldeará y templará a tu adorado juguete para tener una experiencia total HOT sin sobresaltos.Déjate caer por un tobogán de sensaciones y por las sinuosas curvas del nuevo TOBOGANE RABBIT… que está vez te hará también saltar."
2,2,BALLENATO,tu vibrador a distancia con aleta móvil y sumergible,"De las profundidades más húmedas llega BALLENATO, el nuevo vibrador ergonómico de AMANTIS.Este grandioso cetáceo vibrante es poseedor de una colita estructurada que adquiere la forma que tú le des. De tal manera que podrías usarlo: como un potente dildo (dejando la aleta articulada en línea recta), como juguete de doble estimulación (vagina- clítoris / ano- perineo) o como vibrador tipo C (insertando sólo la parte de la aleta y permitiendo la penetración mientras lo llevas puesto)… entre muchas otras formas de estimulación externa.Además, BALLENATO es disfrutón y aparte de pasarlo muy bien en el hogar y recrearse en los juegos húmedos del baño, le encanta dar paseos al aire libre o incluso que lo saques a cenar porque gracias a su mando a distancia, podrás llevarlo puesto y activarlo allá donde tú quieras.Hazte ya las veinte mil leguas de viaje submarino con sus múltiples y diversas inmersiones.¡FREE BALLENATO!"
3,3,TANDEM 2 flex,vibrador doble flexible con mando,"Tanto si ya eras fan de nuestro querido TANDEM como si no lo conocías, te presentamos su nueva versión TANDEM 2 flex: más flexible, reducido, recargable, con dos motores y mando a distancia. Haz tu Tándem ""chica->chica"" o ""chica->chico"" y prepátrate, TANDEM 2 flex lo tiene todo para dar mientras tomas, con un plus de comodidad y vibraciones!Su nuevo diseño –más compacto- lo hace mucho más manejable, sin renunciar a la máxima calidad de su silicona suave y aterciopelada. TÁNDEM 2 flex lleva dos motores, uno para quien lo lleve puesto y otro en el dildo, que vibrarán al ritmo de 10 patrones de vibración diferentes y tres niveles de intensidad distintos, que puedes controlar desde el propio juguete o con… ¡un mando a distancia!Decide si quieres utilizarlo sin arnés o en combinación con un modelo abierto, como nuestro precioso arnés-braguita de amantis, que te aportará un extra de sujeción para no tener que preocuparte por nada. Se adapta a cualquier cuerpo gracias a su flexibilidad y, si deseas darle mayor abertura, ábrelo y cierralo con tus manos varias veces para aumentar su arco antes del primer us

In [14]:
dataframe.to_csv('./Data/productos.csv',header=True,index=False)
df_engineer.to_csv('./Data/tags.csv',header=True,index=False)

## 3.- Adecuamos el campo **Date**.

Este campo en el fichero de comentarios es de tipo *string*, dado que no lo reconoce como date.

Hay que solucionarlo, para lo cual elimineramos el día de la semana del mismo dejando el resto del texto y convirtiéndolo en tipo date.


In [34]:
comentarios=r'\Data\comentarios_scrape.csv'

In [35]:
folder
new_path =folder+comentarios
df_comments=pd.read_csv(new_path,index_col=False)

In [36]:
df_comments.head()

,Unnamed: 0,ID,DATE,RATIO,USERS,COMMENT
0,0,0,"jueves 18 mayo, 2023",5,David,"Muy recomendable. El bote cunde, hidrata muy bien, sirve para todos los juguetes. Incoloro, sin sabor. Puede que un poco pegajoso al principio, pero nada más. Muy contento."
1,1,0,"miércoles 12 abril, 2023",5,Ferran,"Es la primera vez que compro este lubricante y pese a que siempre se pueden descubrir cosas mejores, me parece que este se va a convertir en mi lubricante número 1 diario para usar con mis juguetes. Espero que con las chicas funcione igual de bien."
2,2,0,"lunes 10 abril, 2023",5,Sergio,"He probado varios tipos de lubricante y este de momento es el mejor. Lo he usado con masturbadores, para el pene y mano y para el ano y va perfecto en todo"
3,3,0,"lunes 23 enero, 2023",5,Jaime,Perfecto para embadurnarte el pincelin y meter hasta el último centímetro
4,4,0,"miércoles 11 enero, 2023",5,YASMINA,"Buen producto, tanto para el cuerpo a cuerpo como para lubricar los jueguetes. Es una compra segura y muy acertada."


In [37]:
df_comments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10659 entries, 0 to 10658
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  10659 non-null  int64 
 1   ID          10659 non-null  int64 
 2   DATE        10659 non-null  object
 3   RATIO       10659 non-null  int64 
 4   USERS       10648 non-null  object
 5   COMMENT     10659 non-null  object
dtypes: int64(3), object(3)
memory usage: 499.8+ KB


De la columna *DATE* extraigo los valores de *DAY*, *MONTH* y *YEAR*.

Con la columna *MONTH* hago un mapeo para convertir en el número correspondiente del mes y así poder generar la nueva columna *DATE* con estos  datos.

In [38]:
df_comments['DAY']=df_comments['DATE'].str.split(' ').str.get(1).astype('Int64')
df_comments['MONTH']=df_comments['DATE'].str.split(' ').str.get(2).str.split(',').str.get(0)
df_comments['YEAR']=df_comments['DATE'].str.split(' ').str.get(-1).astype('Int64')
dm_mapping={
    'enero':1, 
    'febrero':2, 
    'marzo':3, 
    'abril':4, 
    'mayo':5,
    'junio':6, 
    'julio':7,
    'agosto':8, 
    'septiembre':9, 
    'octubre':10, 
    'noviembre':11, 
    'diciembre':12,
} 
df_comments['MONTH']=df_comments['MONTH'].map(dm_mapping)
df_comments.head()

,Unnamed: 0,ID,DATE,RATIO,USERS,COMMENT,DAY,MONTH,YEAR
0,0,0,"jueves 18 mayo, 2023",5,David,"Muy recomendable. El bote cunde, hidrata muy bien, sirve para todos los juguetes. Incoloro, sin sabor. Puede que un poco pegajoso al principio, pero nada más. Muy contento.",18,5,2023
1,1,0,"miércoles 12 abril, 2023",5,Ferran,"Es la primera vez que compro este lubricante y pese a que siempre se pueden descubrir cosas mejores, me parece que este se va a convertir en mi lubricante número 1 diario para usar con mis juguetes. Espero que con las chicas funcione igual de bien.",12,4,2023
2,2,0,"lunes 10 abril, 2023",5,Sergio,"He probado varios tipos de lubricante y este de momento es el mejor. Lo he usado con masturbadores, para el pene y mano y para el ano y va perfecto en todo",10,4,2023
3,3,0,"lunes 23 enero, 2023",5,Jaime,Perfecto para embadurnarte el pincelin y meter hasta el último centímetro,23,1,2023
4,4,0,"miércoles 11 enero, 2023",5,YASMINA,"Buen producto, tanto para el cuerpo a cuerpo como para lubricar los jueguetes. Es una compra segura y muy acertada.",11,1,2023


Convierto la columna *DATE* en datetime con los datos de las 3 últimas columnas.

In [39]:
df_comments['DATE'] = pd.to_datetime(df_comments.iloc[:,-3:])
df_comments.tail()

,Unnamed: 0,ID,DATE,RATIO,USERS,COMMENT,DAY,MONTH,YEAR
10654,11546,574,2013-06-27,5,Laura,"Una monada, quedan super sexy y genial para llevar el ritmo mientras le das al temita jeje. Ademas las pinzas ajustables van muy bien por si tienes los pezones muy sensibles. Y vienen en una cagita monisima, parece una mini-bombonera de lujo.",27,6,2013
10655,11547,574,2013-02-01,5,miguel,"Hola este producto lo compramos y ese mismo día lo probamos.Es ideal y el regulador de presión es muy útil. Los cascabeles le dan un toque muy exóticoLo mejor de todo el precio, ya que lo hemos adquirido con una oferta sensacional.Lo recomendamos.",1,2,2013
10656,11548,575,2022-12-22,3,Marta,"Lo malo de este juguete es la bola de la mordaza, prefiero otra de silicona y con agujeros para poder respirar mejor. No obstante, es un juguete muy hot para esos momentos de sumisión.",22,12,2022
10657,11549,575,2018-02-18,4,Héctor,Te limita un poco el movimiento... pero de eso se trata no? está genial,18,2,2018
10658,11550,575,2016-11-14,5,Francisco,"me encanta. En la foto no se aprecia, pero se puede regular la distancia entre las esposas y el cuello. Inmoviliza a tu pareja totalmente, sobre todo en la posición del perrito. Huyyyyyyyyy......",14,11,2016


Cabe la posibilidad de eliminar las 3 últimas columnas. 

Esto lo valoraremos después cuando hagamos visualizaciones.

De momento lo dejaremos comentado.

In [40]:
df_comments=df_comments.iloc[:,:-3]
df_comments.head()

,Unnamed: 0,ID,DATE,RATIO,USERS,COMMENT
0,0,0,2023-05-18,5,David,"Muy recomendable. El bote cunde, hidrata muy bien, sirve para todos los juguetes. Incoloro, sin sabor. Puede que un poco pegajoso al principio, pero nada más. Muy contento."
1,1,0,2023-04-12,5,Ferran,"Es la primera vez que compro este lubricante y pese a que siempre se pueden descubrir cosas mejores, me parece que este se va a convertir en mi lubricante número 1 diario para usar con mis juguetes. Espero que con las chicas funcione igual de bien."
2,2,0,2023-04-10,5,Sergio,"He probado varios tipos de lubricante y este de momento es el mejor. Lo he usado con masturbadores, para el pene y mano y para el ano y va perfecto en todo"
3,3,0,2023-01-23,5,Jaime,Perfecto para embadurnarte el pincelin y meter hasta el último centímetro
4,4,0,2023-01-11,5,YASMINA,"Buen producto, tanto para el cuerpo a cuerpo como para lubricar los jueguetes. Es una compra segura y muy acertada."


In [41]:
df_comments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10659 entries, 0 to 10658
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Unnamed: 0  10659 non-null  int64         
 1   ID          10659 non-null  int64         
 2   DATE        10659 non-null  datetime64[ns]
 3   RATIO       10659 non-null  int64         
 4   USERS       10648 non-null  object        
 5   COMMENT     10659 non-null  object        
dtypes: datetime64[ns](1), int64(3), object(2)
memory usage: 499.8+ KB


In [44]:
df_comments.to_csv('./Data/comentarios.csv',header=True,index=True)           # Tengo que generar el path correcto